WISO100303 / Johannes Schmidt & Peter Regner

# **An introduction to scientific programming**

<br> <br> <br> <br><br> <br> <br> <br>

In [ ]:
COUNTRIES = 'Austria', 'Germany', 'Switzerland', 'Italy', 'Spain', 'Sweden', 'United Kingdom'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Use this for larger plots:
#matplotlib.rc('figure', figsize=(15, 10))
matplotlib.rc('figure', figsize=(10, 8))

# Data
We use a rather extensive data set during this class. Below, there is automatic download code for that data. After each restart of your notebook, [Datalore will download the file again](https://datalore-forum.jetbrains.com/t/how-to-sync-or-copy-files-without-the-web-interface/787/8) if the Datalore kernel is used, which takes time. If you want to spare that time, we therefore recommend to manually upload the data. To do so, please:
- Download the following files from boku box listed in the code cell below
- Click on the paperclick symbol on the left
- Click on the upload button (at the bottom) and choose to upload the file you just downloaded
- Wait until the upload is finished

Note: unfortunately there is a [bug in Datalore](https://datalore-forum.jetbrains.com/t/uploading-from-url-progress-message-does-not-show-completion/856), when using _Upload from URL_. It might work too, but at least the progress message seems to be wrong.

In [ ]:
def download_era5_temperature():
    import os
    import cdsapi

    c = cdsapi.Client()

    filename = 'temperatures_era5.nc'
    north, west, south, east = 70.,-13.5, 35.5, 24.5

    c.retrieve(
        'reanalysis-era5-land',
        {
            'format': 'netcdf',
            'variable': '2m_temperature',
            'area': [
                north, west, south, east
            ],
            'grid': [0.5, 0.5],  # grid in 0.5deg steps in longitude/latitude
            'day': [f"{day:02d}" for day in range(1, 32)],
            'time': [f"{hour:02d}:00" for hour in range(24)],
            'month': [f"{month:02d}" for month in range(1, 13)],
            'year': [str(year) for year in range(2015, 2021)],
        },
        f"{filename}.part")

    # this prevents you from accidentally using broken files:
    os.rename(f"{filename}.part", filename)

# download_era5_temperature()

In [ ]:
# donwload the data
def download_attached_files():
    import urllib
    import os.path
    fnames = {
              'entsoe-demand-shortened.pickle': 'https://bokubox.boku.ac.at/index.php/get/4bbdac7c2872bd8cefd4fb6a4267a879/entsoe-demand-shortened.pickle',
              'temperatures_era5.nc': 'https://bokubox.boku.ac.at/index.php/get/181272eaf961fda5bd240fa70644e400/temperatures_era5.nc',
              #'countries.geojson': 'https://bokubox.boku.ac.at/index.php/get/de2d9a661f31c67f961a7157fdb64781/countries.geojson',
              'country_points.csv': 'https://bokubox.boku.ac.at/index.php/get/9c8f2d8138fa659887a0592f0132f56f/country_points.csv',
    }
    for fname, url in fnames.items():
        if not os.path.exists(fname):
            urllib.request.urlretrieve(url, filename=fname)

download_attached_files()

In [ ]:
def get_hourly_country_data(data, country):
    ret_data = data[data["AreaName"] == country].interpolate() # data may contain NAs, therefore inteprolate
    ret_data = ret_data.resample("1h").mean().interpolate() # not all hours may be  complete 
                                                            # (i.e. some last 15 minutes are lacking, therefore another inpolation here)

    return ret_data

power_demand = pd.read_pickle("entsoe-demand-shortened.pickle")

power_demand = power_demand.loc["2015-01-01":"2019-12-31"]
power_demand_at_hourly = get_hourly_country_data(power_demand, "Austria")

# Temperature data

ERA5 data is provided as NetCDF file. The library `xarray` comes in very handy to load such files.

In [ ]:
import xarray as xr

In [ ]:
temperatures_dataset = xr.open_dataset('temperatures_era5.nc')

The method `open_dataset()` does not load all data into RAM. So it is very fast. No values are read from disk yet.

In [ ]:
temperatures_dataset

In [ ]:
temperatures = temperatures_dataset.t2m

In [ ]:
temperatures

In [ ]:
temperatures.isel(time=0).values

In [ ]:
temperatures.sel(time='2020-03-29 17:00').values

Oh there are NaN values? How many of them?

In [ ]:
total_size = temperatures.sizes['time'] * temperatures.sizes['latitude'] * temperatures.sizes['longitude']
float(np.isnan(temperatures).sum() / total_size)

Uh 55% of missing values.. That's not good! What could that be?

In [ ]:
(~np.isnan(temperatures)).prod(dim='time').plot.imshow(cmap='gray')

**Note:** We downloaded the product `'reanalysis-era5-land'`, there is also `'era5-single-levels'` which contains data also for locations in the sea.

## Exercise 1

Plot the mean temperature in C° for each location!

- Which ways are there to plot the temperature?
- Can you think of at least two different plots?
- Can you use `xarray` directly for plotting?
- Keep an eye on the axes labels! Is this done automatically? If so, how does this work?

Temperature seems not to be in °C...

In [ ]:
temperatures = temperatures - 273.15
temperatures.name = 'Temperature [C°]'

In [ ]:
temperatures.mean(dim='time').plot()

In [ ]:
temperatures.mean(dim=['latitude', 'longitude']).plot()

## Exercise 2

Plot the temperatures at our location.

**Instructions:**
- first use the manual way and try to find the nearest coordinates and then use the `.sel()` method, note that we are using a 0.5° grid
- now directly pass the coordinates and use the parameter `method='nearest'` to get the temperature of the nearest grid point

What was the temperature on the 13th of March 2020 at 14:00?

**Note:** Vienna is approximately at lognitude=16, latitude=48 - not the other way around.

In [ ]:
you_are_here =  np.array([16.357709, 48.232303])

In [ ]:
you_ar

In [ ]:
temperatures.sel(longitude=round(you_are_here[0] * 2)/2, latitude=round(you_are_here[1] * 2)/2).plot()

In [ ]:
temperatures.sel(longitude=you_are_here[0], latitude=you_are_here[1], method='nearest').plot()

In [ ]:
temperatures.sel(longitude=you_are_here[0], latitude=you_are_here[1], time='2020-03-13 14:00', method='nearest')

# Pick random grid points to calculate the mean

As a next step, we want to calculate the mean temperature for each country.

We'll pick just some random samples from the grid for each country, to make computation of the mean faster. The coordinates are already prepared as CSV file, which has been generated using the following code.

In [ ]:
def choose_country_points(longitude, latitude, grid_points_per_country=200):
    """Pick random points for each country from the grid with axis ``longitude`` and ``latitude``.
    ``size`` is the number of points ot be picked for 
    
    Returns a dataframe with two columns per country (longitude & latitude)
    and ``grid_points_per_country`` numbers of rows.
    
    Note: GeoJSON always uses WGS84:
    https://tools.ietf.org/html/rfc7946
    
    """
    # local import to avoid dependency
    import geopandas
    from shapely.geometry import Point
    
    longitudes, latitudes = np.meshgrid(longitude, latitude)
    
    longitudes = longitudes.flatten()
    latitudes = latitudes.flatten()
    
    grid_points = geopandas.GeoSeries(geopandas.points_from_xy(longitudes.flatten(),
                                                           latitudes.flatten()))
    
    # XXX fix me, correct path!
    country_borders = geopandas.read_file('countries.geojson')

    chosen_gridpoints = pd.DataFrame()

    for country in COUNTRIES:
        print(f"Picking grid points for {country}...")
        is_country = country_borders.ADMIN == country

        country_border = country_borders[is_country].geometry.iloc[0]

        is_in_country = grid_points.within(country_border)

        number_of_points = is_in_country.sum()
        
        # make things reproducible!
        np.random.seed(42)
        
        idcs = np.random.randint(number_of_points, size=grid_points_per_country)

        chosen_gridpoints[f'{country}_longitude'] = longitudes[is_in_country][idcs]
        chosen_gridpoints[f'{country}_latitude'] = latitudes[is_in_country][idcs]
        
    return chosen_gridpoints

In order to recreate the `country_points.csv` one needs to install `geopandas` and download a `GeoJSON` file (23MB) which contains the country borders. On windows there might be no `wget` command, use `requests.get()` instead to download the file:

In [ ]:
# !conda install --yes geopandas
# !wget -O ../data/countries.geojson https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson

The following lines create the `country_points.csv`:

In [ ]:
# country_points = choose_country_points(temperatures.longitude, temperatures.latitude)
# country_points.to_csv('../data/country_points.csv', index=False)

But since it is already prepared, let's just load it...

In [ ]:
country_points = pd.read_csv('country_points.csv')

In [ ]:
country_points

In [ ]:
len(country_points)

Let's plote some of these points:

In [ ]:
plt.plot(country_points['Austria_longitude'], country_points['Austria_latitude'], 'o')
plt.xlabel('Longitude [deg]')
plt.ylabel('Latitude [deg]');

In [ ]:
plt.plot(country_points['Germany_longitude'], country_points['Germany_latitude'], 'o')
plt.xlabel('Longitude [deg]')
plt.ylabel('Latitude [deg]');

# Calculate mean temperature for each country

In [ ]:
country = 'Austria'
country_temperature = temperatures.sel(
        longitude=xr.DataArray(country_points['Austria_longitude'], dims='points'),
        latitude=xr.DataArray(country_points['Austria_latitude'], dims='points'))

In [ ]:
country_temperature

In [ ]:
def calc_country_temperature(country):
    country_temperature = temperatures.sel(
        longitude=xr.DataArray(country_points[f'{country}_longitude'], dims='points'),
        latitude=xr.DataArray(country_points[f'{country}_latitude'], dims='points')).mean(dim='points')
    return country_temperature

In [ ]:
temperature_at = calc_country_temperature('Austria')

In [ ]:
temperature_at.plot()

In [ ]:
x# XXX this does not seem to be correct... 2019 had a lower electricity consumption than 2018, so we assume it was very warm.
# But this plot is shifted, there is no 2021 data in there... no time for that! giving up.
temperature_at.resample(time='Y').mean().plot()

# Who likes to have it warm?

In [ ]:
plt.plot(temperature_at.sel(time=power_demand_at_hourly.index),
         power_demand_at_hourly, 'o')
plt.xlabel('Temperature [°C]')
plt.ylabel('Load [MW]');

We can see the U shaped relation between load and temperature, but there is a lot of variation in there. Could we reduce this somehow?

In [ ]:
idcs = (power_demand_at_hourly.index.weekday == 2) & (power_demand_at_hourly.index.hour == 9)
idcs

In [ ]:
plt.plot(temperature_at.sel(time=power_demand_at_hourly.index[idcs]),
         power_demand_at_hourly[idcs], 'o')
plt.ylim(6_000, 11_000)
plt.xlabel('Temperature [°C]')
plt.ylabel('Load [MW]')
plt.title("Load vs Temperature (Wednesdays 9:00am)");

In [ ]:
from scipy.ndimage import median_filter

In [ ]:
power_demand_at_hourly

In [ ]:
power_temperature = pd.DataFrame()
power_temperature['TotalLoadValue'] = power_demand_at_hourly.TotalLoadValue[idcs]
power_temperature['Temperature'] = temperature_at.interp(time=power_demand_at_hourly.index[idcs])

power_temperature = power_temperature.sort_values('Temperature')

#plt.plot(power_temperature.Temperature,
#         power_temperature.TotalLoadValue, '-')

plt.plot(power_temperature.Temperature,
         median_filter(power_temperature.TotalLoadValue,
                                                 mode='nearest',
                                                 size=30),
         '-')

plt.ylim(6_000, 11_000)
plt.xlabel('Temperature [°C]')
plt.ylabel('Load [MW]')
plt.title("Load vs Temperature (Wednesdays 9:00am)");

A `median_filter()` will replace each value by the median of it's surroundings of size `size`:

In [ ]:
median_filter(np.array([1., 1., 1., 1., 5., 1., 1.]), size=3)

In [ ]:
median_filter(np.array([1., 1., 1., 1., 5., 5., 1.]), size=3)

In [ ]:
power_demand

In [ ]:
for country in COUNTRIES:
#for country in ('Austria', 'Germany', 'Switzerland'):
#for country in ('Spain', 'Italy'):
    power_demand_country = power_demand[power_demand.AreaName == country]

    country_temperature = calc_country_temperature(country)

    # select observations from Wednesdays 9:00am
    idcs = (power_demand_country.index.weekday == 2) & (power_demand_country.index.hour == 9)

    power_temperature = pd.DataFrame()

    power_temperature['TotalLoadValue'] = power_demand_country.TotalLoadValue[idcs]
    power_temperature['Temperature'] = country_temperature.interp(time=power_demand_country.index[idcs])
    power_temperature = power_temperature.sort_values('Temperature')

    normalized_load = power_temperature.TotalLoadValue / power_temperature.TotalLoadValue.mean()
    normalized_load_filtered =  median_filter(normalized_load, mode='nearest', size=30)
    
    lines, = plt.plot(power_temperature.Temperature, normalized_load_filtered, '-', label=country)
    
    #if country == 'United Kingdom':
    #    plt.plot(power_temperature.Temperature, normalized_load, 'o-',
    #             linewidth=0.5, markersize=2, alpha=0.4,
    #             color=lines.get_color(),
    #             label=f"{country} (unfiltered)")


plt.xlabel('Temperature [°C]')
plt.ylabel('Load relative to mean load')
plt.legend();